In [31]:
import tensorflow as tf
import numpy as np
import random
import matplotlib
import matplotlib.pyplot as plt
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
import os

In [32]:
# Load pickled data
import pickle

# TODO: Fill this in based on where you saved the training and testing data

training_file = "/Users/johnreid/capstone/jupyter_notebooks/data/train.p"
testing_file = "/Users/johnreid/capstone/jupyter_notebooks/data/test.p"

with open(training_file, mode='rb') as f:
    train = pickle.load(f)
with open(testing_file, mode='rb') as f:
    test = pickle.load(f)
    
X_train, y_train = train['features'], train['labels']
X_test, y_test = test['features'], test['labels']

n_classes = len(np.unique(y_train))

In [33]:
# Normalize the inputs to between 0 and 1
X_train_norm = X_train / 256
X_test_norm = X_test / 256

# Transform into one-hot encoding
def vectorized_result(j, n_classes):
    e = np.zeros(n_classes)
    e[j] = 1.0
    return e

y_train = np.asarray([vectorized_result(i, n_classes) for i in y_train])
y_test = np.asarray([vectorized_result(i, n_classes) for i in y_test])

Now for some dataset augmentation. We'll just do random rotation at 90, 180 and 270 degrees for now

In [34]:
def random_rotation(img):
    num_rotations = random.randint(1, 3)
    return np.rot90(img, num_rotations)

X_train_rot = [random_rotation(x) for x in X_train_norm]

X_train_norm = np.concatenate((X_train_norm, X_train_rot))
y_train = np.concatenate((y_train, y_train))

In [35]:
# Shuffle the data
X_train_norm, y_train = shuffle(X_train_norm, y_train)
X_test_norm, y_test = shuffle(X_test_norm, y_test)

X_train, X_validation, y_train, y_validation = train_test_split(X_train_norm, y_train, 
                                                                test_size=0.10, random_state=42)

# # Add some of the test data to the training data
# X_train_norm = np.concatenate((X_train_norm, X_test_norm[:10791]), axis = 0)
# X_test_norm = X_test_norm[10791:]

# y_train = np.concatenate((y_train, y_test[:10791]), axis = 0)
# y_test = y_test[10791:]

print("X train shape:", X_train.shape)
print("y train shape:", y_train.shape)
print("X validation shape:", X_validation.shape)
print("y validation shape:", y_validation.shape)
print("X test shape:", X_test_norm.shape)
print("ytest shape:", y_test.shape)

X train shape: (62734, 32, 32, 3)
y train shape: (62734, 43)
X validation shape: (15684, 32, 32, 3)
y validation shape: (15684, 43)
X test shape: (12630, 32, 32, 3)
ytest shape: (12630, 43)


In [36]:
## Helper functions
def weight_variable(shape):
    initial = tf.truncated_normal(shape, stddev = 0.1)
    return tf.Variable(initial)

def bias_variable(shape):
    initial = tf.truncated_normal(shape=shape, stddev = 0.1)
    return tf.Variable(initial)

def conv2d(x, W):
    return tf.nn.conv2d(x, W, strides = [1,1,1,1], padding='VALID')

def max_pool_2x2(x):
    return tf.nn.max_pool(x, ksize = [1, 2, 2, 1],
                         strides = [1,2,2,1], padding = 'VALID')

def LeNet_v1(x): 
    W_conv1 = weight_variable([5,5,3,6])
    b_conv1 = bias_variable([6])

    x_image = tf.reshape(x, [-1, 32, 32, 3])

    h_conv1 = tf.nn.relu(conv2d(x_image, W_conv1) + b_conv1)
    h_pool1 = max_pool_2x2(h_conv1)

    W_conv2 = weight_variable([5,5,6, 16])
    b_conv2 = bias_variable([16])

    h_conv2 = tf.nn.relu(conv2d(h_pool1, W_conv2) + b_conv2)
    h_pool2 = max_pool_2x2(h_conv2)

    W_conv3 = weight_variable([5,5,16,400])
    b_conv3 = bias_variable([400])

    h_conv3 = tf.nn.relu(conv2d(h_pool2, W_conv3) + b_conv3)
    h_conv3_flat = tf.reshape(h_conv3, [-1, 400])

    W_fc1 = weight_variable([400, 800])
    b_fc1 = bias_variable([800])
    h_fc1 = tf.nn.relu(tf.matmul(h_conv3_flat, W_fc1) + b_fc1)
    
    h_fc1 = tf.nn.dropout(h_fc1, keep_prob)

    W_fc2 = weight_variable([800, n_classes])
    b_fc2 = bias_variable([n_classes])
    final = tf.matmul(h_fc1, W_fc2) + b_fc2
    
    return final

In [37]:
x = tf.placeholder(tf.float32, (None, 32, 32, 3))
y_ = tf.placeholder(tf.float32, shape=(None))
keep_prob = tf.placeholder(tf.float32)

logits = LeNet_v1(x)

cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=y_))
train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)

correct_prediction = tf.equal(tf.argmax(logits, 1), tf.argmax(y_, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

sess = tf.InteractiveSession()
sess.run(tf.global_variables_initializer())

In [44]:
BATCH_SIZE = 50
EPOCHS = 25
INPUT_SIZE = len(X_train_norm)

def evaluate(X_data, y_data):
    num_examples = len(X_data)
    total_accuracy = 0
    sess = tf.get_default_session()
    for offset in range(0, num_examples, BATCH_SIZE):
        batch_x, batch_y = X_data[offset:offset+BATCH_SIZE], y_data[offset:offset+BATCH_SIZE]
        accuracy_count = sess.run(accuracy, feed_dict={x: batch_x, y_: batch_y, keep_prob: 1.0})
        total_accuracy += (accuracy_count * len(batch_x))
    return total_accuracy / num_examples

def run_lenet(training_images, training_labels, test_images, test_labels, batch_size, epochs, input_size):
    results = []
    for i in range(epochs):
        print("Epoch %s" %i)
        training_images, training_labels = shuffle(training_images, training_labels)
        for j in range(int(input_size / batch_size)):
            batch_xs = training_images[batch_size * j: batch_size*(j+1)]
            batch_ys = training_labels[batch_size * j: batch_size*(j+1)]
            train_step.run(feed_dict = {x:batch_xs, y_:batch_ys, keep_prob: 0.5})
            
        valid_accuracy = evaluate(test_images[:10000], test_labels[:10000])
        print("validation accuracy: %g"% valid_accuracy)
        results.append(valid_accuracy)
    return results

res = run_lenet(X_train, y_train, X_validation, y_validation, BATCH_SIZE, EPOCHS, INPUT_SIZE)

Epoch 0
validation accuracy: 0.9548
Epoch 1
validation accuracy: 0.9589
Epoch 2
validation accuracy: 0.9517
Epoch 3
validation accuracy: 0.951
Epoch 4
validation accuracy: 0.9612
Epoch 5
validation accuracy: 0.9655
Epoch 6
validation accuracy: 0.9515
Epoch 7
validation accuracy: 0.9613
Epoch 8
validation accuracy: 0.9619
Epoch 9
validation accuracy: 0.9651
Epoch 10
validation accuracy: 0.9628
Epoch 11
validation accuracy: 0.9686
Epoch 12
validation accuracy: 0.9619
Epoch 13
validation accuracy: 0.9676
Epoch 14
validation accuracy: 0.966
Epoch 15
validation accuracy: 0.9636
Epoch 16
validation accuracy: 0.969
Epoch 17
validation accuracy: 0.9686
Epoch 18
validation accuracy: 0.9707
Epoch 19
validation accuracy: 0.9529
Epoch 20
validation accuracy: 0.9702
Epoch 21
validation accuracy: 0.9618
Epoch 22
validation accuracy: 0.9709
Epoch 23
validation accuracy: 0.9708
Epoch 24
validation accuracy: 0.9712


In [45]:
1e-4

0.0001

In [46]:
saver = tf.train.Saver()

save_path = "Users/johnreid/Desktop/model.ckpt"

if not os.path.isabs(save_path):
    save_path = os.path.abspath(os.path.join(os.getcwd(), save_path))
save_path2 = saver.save(sess, save_path)
print("Model saved in file: %s" % save_path2)

ValueError: Parent directory of /Users/johnreid/capstone/jupyter_notebooks/Users/johnreid/Desktop/model.ckpt doesn't exist, can't save.

In [47]:
saver = tf.train.Saver(write_version=tf.train.SaverDef.V2)
save_path2 = saver.save(sess, os.path.join(os.getcwd(), 'tmp/trained_variables2.ckpt'))
print("Model saved in file: %s" % save_path2)

ValueError: Parent directory of /Users/johnreid/capstone/jupyter_notebooks/tmp/trained_variables2.ckpt doesn't exist, can't save.